---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
#P1_Graphs
#P1_Graphs[0].edges()

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
#This works but it is a bit rough and ready
def graph_identification():
    clustering=[]
    path=[]
    for G in P1_Graphs:
        #print(G)
        c=nx.average_clustering(G)
        p=nx.average_shortest_path_length(G)
        clustering.append(nx.average_clustering(G))
        path.append(nx.average_shortest_path_length(G))
        
    """
    'PA'   Few nodes have v. high degree, majority have v. low degree (low clust coeff, v. low path length)
    'SW_L' (high clust coeff, low path length)
    'SW_H' High prob randomises the connections creating low clust coeff (low clust coeff,  low path length)
    """
    cmean=np.mean(clustering)
    cmax=np.max(clustering)
    cmin=np.min(clustering)
    cbuffer=(cmax-cmin)/10

    pmean=np.mean(path)
    pmax=np.max(path)
    pmin=np.min(path)
    pbuffer=(pmax-pmin)/10
    #print(clustering)
    #print(path)
    #print(cbuffer)
    #print(pbuffer)
    
    clf=[]
    for i in range(len(clustering)):
        if clustering[i] < cmean-cbuffer and path[i] < pmean-pbuffer:
            clf.append('PA')
        elif clustering[i] < cmean and path[i] < pmean:
            clf.append('SW_H')
        else:
            clf.append('SW_L')  
        
    return clf
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
#The code I used to extract the files from Coursera Jupyter notebook
G = nx.read_gpickle('email_prediction.txt')
df = pd.DataFrame(index=G.nodes())
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_recall_curve, roc_curve, auc, roc_auc_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

df.index.name='Node'
df['degree'] = pd.Series(dict(G.degree()))
df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
df['closeness'] = pd.Series(nx.closeness_centrality(G))
df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True , endpoints=False, k=50))
df['pr'] = pd.Series(nx.pagerank(G, alpha=0.8))
df['hits_hub'] = pd.Series(nx.hits(G, normalized=True)[0])
df['hits_auth'] = pd.Series(nx.hits(G, normalized=True)[1])

df_train=df[~pd.isnull(df['ManagementSalary'])]
df_train_X=df_train.drop(labels='ManagementSalary', axis=1)
df_train_y=df_train['ManagementSalary']

df_test=df[pd.isnull(df['ManagementSalary'])].drop(labels='ManagementSalary', axis=1)

In [6]:
def salary_predictions(train=False):
    #Train test split the data - can use roc_auc to test
    if train==True:
        X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_y, random_state=19)
    else:
    #Cannot use roc_auc with the full data:
        X_train, X_test, y_train = df_train_X, df_test, df_train_y   
        
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    #Be careful not to use s.fit_transform()
    X_test_scaled = s.transform(X_test)
    
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train_scaled, y_train)
    #predict_proba returns a two values [prob of 0, prob of 1]
    test_proba=clf.predict_proba(X_test_scaled)
    answer=pd.Series(test_proba[:,1], X_test.index)
  
    #decision function gives the distance from the dividing line.
    #y_score_lr = clf.decision_function(X_test)
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    
    #false positive rate, true positive rate, thresholds of decision function
    #roc curve takes true value and the score (either decision function or probability.)
    if train==True:
        fpr, tpr, _ = roc_curve(y_test, test_proba[:,1])
        roc_auc = auc(fpr, tpr)
    else:
        roc_auc=('This is not a test!')
    #return (roc_auc, answer)    
    return answer

salary_predictions()

Node
1       0.035103
2       0.960239
5       0.991767
8       0.060699
14      0.101295
18      0.047768
27      0.262345
30      0.213795
31      0.635571
34      0.029372
37      0.018776
40      0.124718
45      0.078706
54      0.224289
55      0.628931
60      0.084187
62      0.992145
65      0.954289
77      0.029081
79      0.168587
97      0.011417
101     0.009844
103     0.233383
108     0.158001
113     0.147500
122     0.006345
141     0.038917
142     0.974205
144     0.114023
145     0.045395
          ...   
913     0.020528
914     0.017992
915     0.018143
918     0.018298
923     0.009500
926     0.016131
931     0.016131
934     0.006620
939     0.009500
944     0.012609
945     0.010019
947     0.034712
950     0.236933
951     0.007339
953     0.022088
959     0.011254
962     0.009500
963     0.184466
968     0.007741
969     0.008698
974     0.013759
984     0.006318
987     0.015411
989     0.010019
991     0.012142
992     0.010019
994     0.012609
996     0

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
for node in G.nodes():
    G.node[node]['community'] = G.node[node]['Department']
    
#possible connections:
n=len(G.nodes())
possible=0.5*n*(n-1)
#print(possible)

df3=pd.DataFrame()
#These calculations take some time!
df3['Jaccard'] = [i[2] for i in list(nx.jaccard_coefficient(G))]
df3['RAI'] = [i[2] for i in list(nx.resource_allocation_index(G))]
df3['Pref Attach'] = [i[2] for i in list(nx.preferential_attachment(G))]
#Using Community
df3['Comm CN'] = [i[2] for i in list(nx.cn_soundarajan_hopcroft(G))]
df3['Comm RA'] = [i[2] for i in list(nx.ra_index_soundarajan_hopcroft(G))]
df3.index=[(i[0], i[1]) for i in nx.preferential_attachment(G)]

df3=df3.join(future_connections, how='outer')

df3_train=df3[~pd.isnull(df3['Future Connection'])]
df3_train_X=df3_train.drop(labels='Future Connection', axis=1)
df3_train_y=df3_train['Future Connection']

df3_test=df3[pd.isnull(df3['Future Connection'])].drop(labels='Future Connection', axis=1)

In [9]:
def new_connections_predictions(train=False):
    #Train test split the data - can use roc_auc to test
    if train==True:
        X_train, X_test, y_train, y_test = train_test_split(df3_train_X, df3_train_y, random_state=19)
    else:
    #Cannot use roc_auc with the full data:
        X_train, X_test, y_train = df3_train_X, df3_test, df3_train_y   
        
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    #Be careful not to use s.fit_transform() on test data
    X_test_scaled = s.transform(X_test)
    
    clf = GradientBoostingClassifier(random_state = 0)
    clf.fit(X_train_scaled, y_train)
    #predict_proba returns two values [prob of 0, prob of 1]
    test_proba=clf.predict_proba(X_test_scaled)
    answer=pd.Series(test_proba[:,1], X_test.index)
  
    #decision function gives the distance from the dividing line.
    #y_score_lr = clf.decision_function(X_test)
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    
    #false positive rate, true positive rate, thresholds of decision function
    #roc curve takes true value and the score (either decision function or probability.)
    if train==True:
        fpr, tpr, _ = roc_curve(y_test, test_proba[:,1])
        roc_auc = auc(fpr, tpr)
    else:
        roc_auc=('This is not a test!')
    return answer

In [10]:
new_connections_predictions(train=False)

(0, 9)          0.027418
(0, 19)         0.057918
(0, 20)         0.218393
(0, 35)         0.018372
(0, 38)         0.016014
(0, 42)         0.857956
(0, 43)         0.014902
(0, 50)         0.013727
(0, 53)         0.565254
(0, 54)         0.038049
(0, 62)         0.339745
(0, 63)         0.054883
(0, 69)         0.027411
(0, 72)         0.012980
(0, 83)         0.174868
(0, 90)         0.026179
(0, 91)         0.017977
(0, 95)         0.107150
(0, 100)        0.027418
(0, 106)        0.983333
(0, 108)        0.018377
(0, 109)        0.012980
(0, 110)        0.012980
(0, 118)        0.015665
(0, 122)        0.026179
(0, 131)        0.037130
(0, 133)        0.722474
(0, 140)        0.060150
(0, 149)        0.118733
(0, 151)        0.016014
                  ...   
(988, 989)      0.012980
(988, 996)      0.012980
(988, 997)      0.046346
(988, 1000)     0.012980
(988, 1002)     0.012980
(989, 994)      0.012980
(989, 996)      0.012980
(989, 1003)     0.012980
(989, 1004)     0.012980
